In [166]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from numba import jit

In [195]:
@jit(nopython=True)
def get_slices_original(image: np.ndarray, slice_width: int, slice_height: int, overlapped_ratio: float) -> list[np.ndarray]:
  window = []
  h, w = image.shape[:2]
  stepSizeY = int(slice_height - slice_height*overlapped_ratio)
  stepSizeX = int(slice_width - slice_width*overlapped_ratio)
  for y in range(0, image.shape[0], stepSizeY):
    for x in range(0, image.shape[1], stepSizeX):
      if w-x < stepSizeX and h-y < stepSizeY:
        window.append(image[h-slice_height:h, w-slice_width:w, :])

      elif w-x < stepSizeX:
        window.append(image[y:y + slice_width, w-slice_width:w, :])

      elif h-y < stepSizeY:
        window.append(image[h-slice_height:h, x:x + slice_height,:])

      else:
        window.append(image[y:y + slice_width, x:x + slice_height,:])

  return window

In [196]:
@jit(nopython=True)
def get_slices_new(image: np.ndarray, slice_width: int, slice_height: int, overlapped_ratio: float) -> list[np.ndarray]:
    h, w = image.shape[:2]
    stepSizeY = int(slice_height - slice_height*overlapped_ratio)
    stepSizeX = int(slice_width - slice_width*overlapped_ratio)
    a = np.arange(0,h, stepSizeX)
    a[-1] = h - slice_height
    b = np.arange(0,w, stepSizeY)
    b[-1] = w - slice_width
    lst = [image[i:i+slice_height, j:j+slice_width, :] for i in a for j in b]
    return lst

In [204]:
import timeit
import random
random.seed(0)
%timeit -n 500 get_slices_original(np.random.randint(0,255,(random.randint(800,1500), random.randint(800,1500),3)), slice_width=300 , slice_height=300, overlapped_ratio=0.1)

random.seed(0)
%timeit -n 500 get_slices_new(np.random.randint(0,255,(random.randint(800,1500), random.randint(800,1500),3)), slice_width=300 , slice_height=300, overlapped_ratio=0.1)

19.1 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 500 loops each)
18.9 ms ± 620 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
